In [637]:
from typing import Union, List
from datetime import datetime
import json

Nel principio Iddio creò i cieli e la terra. E la terra era informe e vuota, e le tenebre coprivano la faccia dell’abisso, e lo spirito di Dio aleggiava sulla superficie delle acque. E Dio disse: Sia la luce! E la luce fu. E Dio vide che la luce era buona; e Dio separò la luce dalle tenebre. E Dio chiamò la luce "giorno", e le tenebre "notte". Così fu sera, poi fu mattina: e fu il primo giorno.

In [638]:
""" Art. 1. 

 (Capacità giuridica). 

 La capacità giuridica si acquista dal momento della nascita. 

 I diritti che la legge riconosce a favore del concepito sono subordinati all'evento della nascita. 

((IL D.LGS. LUOGOTENENZIALE 14 SETTEMBRE 1944, N. 287 HA CONFERMATO L'ABROGAZIONE DEL PRESENTE COMMA)).  """

class Soggetto:
    def __init__(self, CF: str):
        self.CF = CF

    def __str__(self) -> str:
        return f'{self.CF}'

    def to_dict(self):

        return {'CF': self.CF}
    
class SoggettoFisico(Soggetto):
    pass

class SoggettoGiuridico(Soggetto):
    pass

""" Art. 810. codice civile

 (Nozione). 

 Sono beni le cose che possono formare oggetto di diritti. 
 """

class Bene:
    def __init__(self, descrizione: str, tipo: str = "giuridico", valore: int = None, quota: int = 100, divisibile:bool = False):
        self.descrizione = descrizione  # Descrizione del bene per una migliore identificazione
        self.valore = valore*quota if quota != 100 else quota
        self.tipo = tipo #mobile/immobile/immateriale
        self.quota = quota / 100
        self.divisibile = divisibile

    def __str__(self):
        return f"{self.descrizione}, Valore: {self.valore}, Tipo: {self.tipo}, Quota: {self.quota}% {self.divisibile if self.quota else ''}."

    def to_dict(self):
        """Return a dictionary representation of the object."""
        return {
            'descrizione': self.descrizione,
            'valore': self.valore,
            'tipo': self.tipo,
            'quota':self.quota,
            'divisibile': self.divisibile
        }

""" Art. 1173. codice civile

 (Fonti delle obbligazioni). 

 Le obbligazioni derivano da contratto, da fatto illecito, o da ogni altr atto o fatto idoneo a produrle in conformità dell'ordinamento giuridico. 

Art. 1174. codice civile

 (Carattere patrimoniale della prestazione). 

 La prestazione che forma oggetto dell'obbligazione deve essere suscettibile di valutazione economica e deve corrispondere a un interesse, anche non patrimoniale, del creditore

Art. 1175. codice civile

 (Comportamento secondo correttezza). 

 Il debitore e il creditore devono comportarsi secondo le regole della correttezza, ((...)).

 """

class Prestazione:
    def __init__(self, descrizione: str, valore: int, tipo: str, soggetto_passivo: Union['SoggettoFisico', 'SoggettoGiuridico'], fungibile:bool = False):
        self.descrizione = descrizione  # Descrizione della prestazione per chiarire l'oggetto
        self.valore = valore # art. 1174
        self.tipo = tipo # dare, fare, ricevere, promettere...
        self.soggetto_passivo = soggetto_passivo #art. 1175
        self.fungibile = fungibile

    def __str__(self):
        return f"{self.descrizione}, Valore: {self.valore}, Tipo: {self.tipo}, Soggetto passivo: {self.soggetto_passivo.CF} {self.fungibile}."
    
    def to_dict(self):
        """Return a dictionary representation of the object."""
        return {
            'descrizione': self.descrizione,
            'valore': self.valore,
            'tipo': self.tipo,
            'soggetto_passivo': self.soggetto_passivo,
            'fungibile': self.fungibile
        }



In [639]:
class Ordinamento:
    def __init__(self, territorio: str = 'italiano', cittadinanza: str = 'italiana'):
        self.territorio = territorio
        self.time = datetime.now()
        self.cittadinanza = cittadinanza
    def __str__(self):
        return f"Repubblica {self.cittadinanza} del popolo {self.territorio}"


class Giudice:
    def __init__(self, soggetto: SoggettoFisico):
        self.CF = soggetto.CF


In [640]:
from typing import Union

class DirittoSoggettivo:
    def __init__(self, titolare: Union['SoggettoFisico', 'SoggettoGiuridico'], 
                 oggetto: Union['Bene', 'Prestazione'],  # Ora è un singolo oggetto, non una lista
                 ordinamento_giuridico: Ordinamento = Ordinamento(territorio='Italiano', cittadinanza='italiana'),
                 soggetto_passivo: Union['SoggettoFisico', 'SoggettoGiuridico', str] = None,  
                 titolo: str = 'legge'):
        self.titolare = titolare
        self.oggetto = oggetto  # Variabile rinominata e modificata per essere un singolo oggetto
        self.ordinamento_giuridico = ordinamento_giuridico
        self.titolo = titolo
        self.soggetto_passivo = soggetto_passivo if soggetto_passivo else [],
        self.categoria = self.determina_categoria()
        self.natura = self.determina_natura()
        self.valore = self.get_valore()
        self.pretesa = self.determina_pretesa()  # Direttamente la descrizione dell'oggetto

    def get_valore(self):
        # Si accede direttamente al valore dell'oggetto
        return self.oggetto.valore if hasattr(self.oggetto, 'valore') and self.oggetto.valore is not None else 0

    def determina_natura(self):
        natura = 'patrimoniale' if self.get_valore() > 0 else 'non patrimoniale'
        if self.soggetto_passivo:
            return ('relativo', natura)
        return ('assoluto', natura)

    def determina_categoria(self):
        if isinstance(self.oggetto, Bene):
            return ('reale', 'di godimento' if self.soggetto_passivo else 'di proprietà')
        elif isinstance(self.oggetto, Prestazione):
            if self.oggetto.soggetto_passivo:
                self.soggetto_passivo.append(self.oggetto.soggetto_passivo)
            return ('di credito',)
        return ('non specificata',)

    def determina_pretesa(self):
        pretesa = []
        if isinstance(self.oggetto, Bene) and self.categoria[0] == 'reale':
            if self.categoria[1] == 'di proprietà' and self.natura[0] == 'assoluto' :
                pretesa.append(f'godimento e disposizione pieno e indisturbato di {self.oggetto.descrizione}')
            elif self.categoria[1] == 'di godimento' and self.natura[0] == 'relativo':
                pretesa.append(f'godimento e disposizione pieno e indisturbato di {self.oggetto.descrizione} nei limiti pattuiti con {self.soggetto_passivo} in {self.titolo}')
        elif isinstance(self.oggetto, Prestazione) and self.categoria[0] == 'di credito':
            pretesa.append(f'prestazione dovuta a {self.soggetto_passivo} a {self.oggetto.tipo} {self.oggetto.descrizione} come stabilito da {self.titolo}')

        return tuple(pretesa)
            
    
    def __str__(self):
        oggetto_desc = str(self.oggetto)
        categoria_desc = ', '.join(self.categoria)
        natura_desc = ', '.join(self.determina_natura())
        soggetto_passivo_desc = str(self.soggetto_passivo) if self.soggetto_passivo else "Nessuno specifico"
        return (f"Diritto {natura_desc} {categoria_desc} di {self.titolare}\n"
                f"Oggetto: {oggetto_desc}\n"
                f"Categoria: {categoria_desc}\n"
                f"Pretesa: {self.pretesa}\n"
                f"Soggetto Passivo: {soggetto_passivo_desc}\n"
                f"Ordinamento Giuridico: {self.ordinamento_giuridico.territorio}\n"
                f"Titolo: {self.titolo}")
        
    def to_dict(self):
        """Return a dictionary representation of the object."""
        return {
            'natura': self.determina_natura(),
            'categoria': self.categoria,
            'titolare': self.titolare.to_dict() if hasattr(self.titolare, 'to_dict') else str(self.titolare),
            'pretesa': self.pretesa,
            'oggetto': self.oggetto.to_dict() if hasattr(self.oggetto, 'to_dict') else str(self.oggetto),
            'soggetto_passivo': self.soggetto_passivo.to_dict() if self.soggetto_passivo else 'erga omnes',
            'titolo': self.titolo,
            'valore': self.valore,
            'ordinamento_giuridico': {'territorio': self.ordinamento_giuridico.territorio} if hasattr(self.ordinamento_giuridico, 'territorio') else str(self.ordinamento_giuridico)
        }

class InteresseLegittimo:
    pass

REGIO DECRETO 30 gennaio 1941, n. 12

In [641]:

""" Art. 2.

(Del pubblico ministero).

Presso la corte di cassazione, le corti di appello, i tribunali ordinari e i tribunali per i minorenni è costituito l'ufficio del pubblico ministero.(109)((110a))
 """
 
class PubblicoMinistero:
    def __init__(self):
        pass

def costituisci_PM(**args):
    return PubblicoMinistero(**args)

In [642]:
""" 
Art. 1.

Dei giudici.

La giustizia nelle materie civile e penale è amministrata:
a) dal giudice di pace;
b) ((LETTERA SOPPRESSA DAL D.LGS. 19 FEBBRAIO 1998, N. 51)); ((110a))
c) dal tribunale ordinario;
d) dalla corte di appello;
e) dalla Corte di cassazione;
f) dal tribunale per i minorenni;
g) dal megistrato di sorveglianza;
h) dal tribunale di sorveglianza; (106a) """

class GiudiceDiPace:
    def __init__(self):
        pass
    pass
class Tribunale:
    def __init__(self):
        self.PM = costituisci_PM()
        pass
    pass
class CortedAppello:
    def __init__(self):
        self.PM = costituisci_PM()
        pass
    pass
class CortediCassazione:
    def __init__(self):
        self.PM = costituisci_PM()
        pass
    pass
class TribunaleperiMinorenni:
    def __init__(self):
        self.PM = costituisci_PM()
        pass
    pass
class MagistratodiSorveglianza:
    def __init__(self):
        pass
    pass
class TribunalediSorveglianza:
    def __init__(self):
        pass
    pass


REGIO DECRETO 28 ottobre 1940, n. 1443

LIBRO PRIMO - DISPOSIZIONI GENERALI

    TITOLO I - DEGLI ORGANI GIUDIZIARI

        CAPO I - Del giudice

            Sezione I - Della giurisdizione e della competenza in generale

In [643]:
class Domanda:
    def __init__(self, oggetti: List[Union['DirittoSoggettivo', 'InteresseLegittimo']], data: datetime, attore: Union['SoggettoFisico', 'SoggettoGiuridico'], controparte: Union['SoggettoFisico', 'SoggettoGiuridico']):
        self.oggetti = [oggetto for oggetto in oggetti] # Lista di diritti soggettivi e/o interessi legittimi
        self.data = data
        self.attore = attore
        self.controparte = controparte
        self.valore = self.get_valore()  # Calcolo iniziale del valore della domanda

    def get_valore(self):
        """Determines the total value of the demand based on specific criteria."""
        valore_totale = 0
        for oggetto in self.oggetti:
            # Check if the object is of type DirittoSoggettivo and its nature includes 'patrimoniale'
            if isinstance(oggetto, DirittoSoggettivo) and 'patrimoniale' in oggetto.natura[1]:
                valore_totale += oggetto.valore
        return valore_totale


    def __str__(self):
        oggetti_desc = '\n'.join([json.dumps(oggetto.to_dict()) for oggetto in self.oggetti])
        return (f"Domanda presentata il {self.data.strftime('%d/%m/%Y')} da {self.attore} contro {self.controparte}, "
                f"comprendente i seguenti diritti: \n {oggetti_desc}. \n Valore totale: €{self.valore}")




In [644]:
class GiurisdizioneOrdinaria:
    
    """  Art. 1. 
    (Giurisdizione dei giudici ordinari). 

    La giurisdizione civile, salvo speciali disposizioni di legge, è esercitata dai giudici ordinari secondo le norme del presente codice. 
         """
    
    def __init__(self, ordinamento: Ordinamento, domanda:Domanda):
        self.ordinamento = ordinamento
        
        """ Art. 5. 
    (Momento determinante della giurisdizione e della competenza). 

    La giurisdizione e la competenza si determinano con riguardo alla legge vigente e allo stato di fatto esistente al momento della proposizione della domanda, e non hanno rilevanza rispetto ad esse i successivi mutamenti della legge o dello stato medesimo. 
        """
        self.domanda = domanda
        self.timestamp = self.ordinamento.time
        self.giurisdizione = ()
        self.competenza = ()
        
            
    def check_giurisdizione(self):
 
        if all(hasattr(oggetto, 'ordinamento_giuridico') for oggetto in self.domanda.oggetti):  
            
            if all(oggetto.ordinamento_giuridico.territorio == self.ordinamento.territorio for oggetto in self.domanda.oggetti):
                self.giurisdizione = (True, self.ordinamento.cittadinanza)
            else:
                self.giurisdizione = (False)
                ordinamenti = set(oggetto.ordinamento_giuridico.cittadinanza for oggetto in self.domanda.oggetti)
                eccezioni = [ordinamento for ordinamento in ordinamenti if ordinamento != self.ordinamento.territorio]
                return False, f'Difetto assoluto di giurisdizione - Giurisdizione estera ({", ".join(eccezioni)})'

            
            if all(isinstance(oggetto, DirittoSoggettivo) for oggetto in self.domanda.oggetti) and self.giurisdizione[0] == True:
                return True, f'Giurisdizione presente - Giurisdizione ordinaria ({self.ordinamento.cittadinanza})'

            else:
                self.giurisdizione = (False, self.ordinamento.cittadinanza)
                return False, 'Difetto relativo di giurisdizione - Giurisdizione speciale'
        else:
            self.giurisdizione = (False, 'inesistente') 
            return False, 'Difetto assoluto di giurisdizione - Giurisdizione inesistente'
    
   
    """ Art. 6. 
        (Inderogabilità convenzionale della competenza). 

        La competenza non può essere derogata per accordo delle parti, salvo che nei casi stabiliti dalla legge. 
    """
    
    def check_competenza(self, accordo = False):
        """  Art. 7.
            (Competenza del giudice di pace).

            Il giudice di pace è competente per le cause relative a beni mobili di valore non superiore a diecimila euro, quando dalla legge non sono attribuite alla competenza di altro giudice. (171) ((173))
            Il giudice di pace è altresì competente per le cause di risarcimento del danno prodotto dalla circolazione di veicoli e di natanti, purché il valore della controversia non superi venticinquemila euro. (171) ((173))

            E' competente qualunque ne sia il valore:
            1) per le cause relative ad apposizione di termini ed osservanza delle distanze stabilite dalla legge, dai regolamenti o dagli usi riguardo al piantamento degli alberi e delle siepi;
            2) per le cause relative alla misura ed alle modalità d'uso dei servizi di condominio di case;
            3) per le cause relative a rapporti tra proprietari o detentori di immobili adibiti a civile abitazione in materia di immissioni di fumo o di calore, esalazioni, rumori, scuotimenti e simili propagazioni che superino la nomale tollerabilità;
            3-bis) per le cause relative agli interessi o accessori da ritardato pagamento di prestazioni previdenziali o assistenziali;
            (72) """
        valore = self.domanda.valore
        beni = [oggetto for oggetto in self.domanda.oggetti if isinstance(oggetto.oggetto, Bene)]
        prestazioni = [oggetto for oggetto in self.domanda.oggetti if isinstance(oggetto.oggetto, Prestazione)]
        
        art7c1_1 = all(bene.oggetto.tipo == 'mobile' for bene in beni) if beni else False
        
        art7c1_2 = any(
            'risarcimento' in prestazione.oggetto.descrizione.lower() and
            any(term in prestazione.oggetto.descrizione.lower() for term in ['autoveicoli', 'natanti'])
            for prestazione in prestazioni
        )
        
        art7c2 = any(
            any(term in prestazione.oggetto.descrizione.lower() for term in ['termini', 'distanze', 'immissioni', 'servizi di condominio', 'previdenziali'])
            for prestazione in prestazioni
        )
        
        art9c2 = any(
        any(term in prestazione.oggetto.descrizione.lower() for term in ['imposta', 'tassa', 'status', 'capacità', 'querela di falso', 'esecusione forzata'])
        for prestazione in prestazioni
            )
        
        if (valore < 10000 and art7c1_1) or (valore < 25000 and art7c1_2) or art7c2:
            self.competenza = GiudiceDiPace()
            """ Art. 9.
            (Competenza del tribunale).
            Il tribunale è competente per tutte le cause che non sono di competenza di altro giudice.
            Il tribunale è altresì esclusivamente competente per le cause in materia di imposte e tasse, per quelle relative allo stato e alla capacità delle persone e ai diritti onorifici, per la querela di falso, per l'esecuzione forzata e, in generale, per ogni causa di valore indeterminabile
                """
            
        elif art9c2:
            self.competenza = Tribunale()
        else:
            self.competenza = Tribunale()
    

        return self.competenza
        
        


In [648]:
ordinamento_ita = Ordinamento(territorio='italiano')
ordinamento_de = Ordinamento(territorio='tedesco', cittadinanza='tedesca')
giudice = Giudice(SoggettoFisico('Irnerio'))
attore = SoggettoFisico('Tizio')
controparti = [SoggettoFisico('Caio'), SoggettoFisico('Sempronio')]
orologio = Bene(descrizione= 'Rolex', valore=1999, tipo='mobile')
borsa = Bene(descrizione= 'Gucci', valore=2000, tipo='mobile')
villa = Bene(descrizione='villa', valore = 10000, tipo='immobile')
#pagamento_corrispettivo_borsa=Prestazione(descrizione=borsa, soggetto_passivo=controparti[0], )  
diritto1 = DirittoSoggettivo(titolare=attore, oggetto=villa, titolo='accordo di servitù', soggetto_passivo=controparti[0], ordinamento_giuridico=ordinamento_ita)
diritto2 = DirittoSoggettivo(titolare=attore, oggetto=borsa, titolo='contratto di compravendita', ordinamento_giuridico=ordinamento_de)

domanda = Domanda(oggetti=[diritto1], data=datetime.now(), attore=attore, controparte=controparti[0])
giurisdizione = GiurisdizioneOrdinaria(ordinamento=ordinamento_ita, domanda=domanda)

print(giurisdizione.check_giurisdizione())


(True, 'Giurisdizione presente - Giurisdizione ordinaria (italiana)')
